##### Chapter 2: Training Simple Machine Learning Algorithms for Classification

We will use 2 of the first described ML algorithms, the perceptron and adaptive linear neaurons. We will implement a perceptron to classify the flowers in the Iris dataset into species classes. Basics of optimization will then be explored using adaptive linear neaurons to lay the ground work for more sophisticated classifiers provided by scikit-learn.

### Formal definition of an artificial neauron

We put the idea of an artificial neauron into context using a binary classifcation where there are 2 classes, 0 and 1. We define a decision function $\sigma (z)$ that takes a linear combination of certain input values $x$, and a corrsonding weight vector $w$ where $z$ is the so called net input $z = w_1x_1 + w_2x_2 + ... + w_mx_m$

\begin{equation}
    w = \begin{bmatrix} W_1 
        \\ . 
        \\ . 
        \\ . 
        \\ W_m 
        \end{bmatrix} , x = \begin{bmatrix} X_1 \\ . \\ . \\ . \\ X_m \end{bmatrix}
\end{equation}



If the net input of a particular example $x^{i}$ is greater than the defined threshold $\theta$ we predict the class 1, otherwise 0. In the perceptron algorithm the decision function $\sigma (.)$ is a variant of the unit step function:

\begin{equation}
    \sigma (z) = \begin{cases}
        1 & if\ z >= \theta \\
        0 & otherwise
        \end{cases}
\end{equation}

To simplify code implimentation later we modify this sllightly by bringing $\theta$ across to the left:
$$
    z >= \theta \\ z - \theta >= 0
$$
Then we define a unit bias $b = -\theta$ and include it in the net input:
$$
z = w_1x_1 + ... + w_mx_m + b = \bf{w}^{T}\bf{x}+b
$$
Third, given the introduction of the unit bias and redefinition of the net input $z$ above we can redefine the decision function as follows:
$$
\sigma (z) = \begin{cases} 1 & if\ z>=0\\0 & otherwise \end{cases}
$$

<p style="text-align: center;">
<img src="Figures\02_02.png" width="800" height="400">
</p>
<p style="text-align: center;"><b>Figure 2.2: A thrshold function that produces a linear decision boundary for a binary classification</b></p>

Figure 2.2 shows how the net input $z = \bf{w}^{T}\bf{x} + b$ is swuashed into a binary 1 or 0 output by the decision function of the perceptron (left) and how it can be used to discriminate between 2 classes seperated by a linear decision boundary (right subfigure).

### The perceptron learning rule

The whole idea is to use a reductionist approach to mimic how a single neauron in the brain works, it either fires or it doesn't. The perceptron algorithm can be summerized by the following steps

1. Initialise the weights and bias unit to 0 or small random numbers
2. For each training example, $\bf{x}^{(i)}$:
 * Compute the output value, $\hat{y}^{(i)}$
 * Update the weight and bias units

 Here the output value is the class label predicted by the Heviside function and the simultaneous update of the bias unit and each weight can be formally written as:

$$
w_j := w_j + \Delta w_j \\ b := b + \Delta b
$$
The update values "deltas" are computed as:
$$
\Delta w_j = \eta(y^{(i)} - \hat{y}^{(i)})x_j^{(i)}\\
\Delta b = \eta(y^{(i)} - \hat{y}^{(i)})
$$
Each weight is associated with a feature $x_j& in the dataset which is used in calculating the correction to $w_j$. $\eta$ is the learning rate, typically a value $\epsilon [0.0, 1.0]$, $y^{(i)}$ is the true class label of the ith training example and $\hat{y^{(i)}}$ is the predicted class. All the updates are done simultaneously meaining we dont recompute the predicted label before the bias unit and all weights are updated.

The convergence of the perceptron is not guarenteed however, if the two classes are not linearly seperable then a clean straight line cannot be drawn between them and the weights and biases will not converge to a constant value because there will always be some training examples that get mislabelled, and therefore the weights and biases will always get updated by some non-zero value. Figure 2.3 shows this diagramatically.


<p style="text-align: center;">
<img src="Figures\02_03.png" width="800" height="400">
</p>
<p style="text-align: center;"><b>Figure 2.3: Example of a linearly seperable problem and 2 non-linearly seperable problems</b></p>

If 2 classes are not linearly seperable then we can define either a max number of epochs to iterate and train over or a max number of acceptable mislabelled training examples, doing so for non-linearly seperable classes will ensure that the algorithm will converge or cease training even if it sees there are still some mislabelled training examples. We will later see an example algorithm call the Adaline algorithm which achieves convergence even when the dataset looks like subfigures 2.3a or 2.3b. There are more advanced algorithms which can produce non-linear decision boundaries which will also be covered later.

<p style="text-align: center;">
<img src="Figures\02_04.png" width="800" height="400">
</p>
<p style="text-align: center;"><b>Figure 2.4: Summary of the learning process for the perceptron</b></p>

### Implementing the perceptron in python 3.9

We will implement a perceptron tailored around the iris dataset touched upon in chapter 1.

### An object oriented perceptron API

We will define the perceptron interface as a python class which allows new perceptrons objects to be initialised that can learn from the data via a fit method and make predictions via a seperate predict method. As a convention we append an underscore (\_) ti attributes that are not created upon initialisation of the object, but we do this by calling the objects other methods, for example self.w_.

In [1]:
import numpy as np

class Perc:
    """
    Perceptron Classifier.

    Parameters
    ----------
    eta : float
        Learning rate (between 0.0 and 1.0)
    n_iter : int
        number of passes over the training dataset
    random_state: int
        Random number generation seed for random weight initialisation

    Attributes
    ----------
    w_ : 1d-array
        Weights after fitting
    b_ : Scalar
        Bias unit after fitting
    
    errors_ : list
        Number of misclassifications (updates) in each epoch

    """

    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state
    
    def fit(self, X, y):
        """
        Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_examples, n_features]
            Training vectors, where n_examples is the number of examples and n_features is the number of features
        y : array-like, shape = [n_examples]
            Target values
        
        Returns
        -------
        self : object

        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=X.shape[1])
        self.b_ = np.float_(0.)
        self.errors_ = []
        for i in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.predict(xi))
                self.w_ += update * xi
                self.b_ += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    def net_input(self, X):
        """Calculate the net input"""
        return np.dot(X, self.w_) + self.b_
    
    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.net_input(X) >= 0.0, 1, 0)